In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from apyori import apriori

In [2]:
data = pd.read_excel("../Datasets/HVAC.xlsx", "HISTORICO_DATOS")

In [3]:
# Eliminando variables inncesarias
variables_a_eliminar = ["C_O_P_ BOMBA CALOR FELIPE", "C_O_P_ BOMBA CALOR CARLOS",
                        "C_O_P_ INSTALACIÓN GRUPO FRÍO 1", "C_O_P_ INSTALACÍON GRUPO FRÍO 2",
                        "ORDEN", "VÁLVULA BY PASS SECUNDARIO FRÍO",
                        "TEMPERATURA CONTROL DE BY PASS SECUNDARIO", "SECUNDARIO FRÍO 1",
                        "SECUNDARIO FRÍO 2", "SECUNDARIO FRÍO 3", "MODO INVIERNO BC1", 
                        "MODO INVIERNO BC2", "PERIODO P6", "CONTROL CALOR", 
                        "CAPACIDAD BOMBA CALOR FELIPE %", "CAPACIDAD BOMBA CALOR CARLOS %",
                        "CAPACIDAD GRUPO DE FRÍO 1"," CAPACIDAD GRUPO DE FRÍO 2",
                        "IMPULSIÓN SECUNDARIO CALOR", "SECUNDARIO CALOR 1", 
                        "SECUNDARIO CALOR 2", "SECUNDARIO CALOR 3"]
lista_variables = data.columns.values.tolist()
subLista = [x for x in lista_variables if x not in variables_a_eliminar]
data = data[subLista]
# Agregando columnas de potencia termica
data["POTENCIA TERMICA BOMBA CALOR CARLOS"] = data["KILO CALORÍAS GENERADAS BOMBA CALOR CARLOS"] * 0.001163
data["POTENCIA TERMICA BOMBA CALOR FELIPE"] = data["KILO CALORÍAS GENERADAS BOMBA CALOR FELIPE"] * 0.001163
data["POTENCIA TERMICA GRUPO FRIO 1"] = data["KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 1"] * 0.001163
data["POTENCIA TERMICA GRUPO FRIO 2"] = data["KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 2"] * 0.001163
data.head()

,Fecha- hora de lectura,POTENCIA TRAFO 2,POTENCIA TRAFO 3,POTENCIA TRAFO 4,POTENCIA TRAFO 5,POTENCIA MEDIA CONECTADA,CONTROL FRÍO,POTENCIA BOMBA CALOR FELIPE,POTENCIA BOMBA CALOR CARLOS,TEMPERATURA AMBIENTE BOMBA CALOR CARLOS,...,TEMPERATURA SALIDA BOMBA CALOR CARLOS,TEMPERATURA SALIDA BOMBA CALOR FELIPE,KILO CALORÍAS GENERADAS BOMBA CALOR CARLOS,KILO CALORÍAS GENERADAS BOMBA CALOR FELIPE,KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 1,KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 2,POTENCIA TERMICA BOMBA CALOR CARLOS,POTENCIA TERMICA BOMBA CALOR FELIPE,POTENCIA TERMICA GRUPO FRIO 1,POTENCIA TERMICA GRUPO FRIO 2
0,2016-01-01 00:00:00,0.783603,38.338665,232.182602,235.755234,557.290039,26.984179,7.253334,7.061334,17.268667,...,36.190666,36.192665,28599.992188,26129.994141,4400.000488,29349.998047,33.261791,30.389183,5.117201,34.134048
1,2016-01-01 00:15:00,0.926969,38.338665,197.970230,113.384056,372.547760,26.805664,0.800000,0.480000,18.049334,...,33.625999,32.770664,163366.671875,193396.765625,18550.000000,0.000000,189.995439,224.920438,21.573650,0.000000
2,2016-01-01 00:30:00,0.793163,38.455196,211.676941,236.968918,469.299347,26.055836,0.800000,0.480000,17.046000,...,33.247997,32.559998,103306.765625,137973.359375,218650.000000,96750.000000,120.145768,160.463017,254.289950,112.520250
3,2016-01-01 00:45:00,0.860073,38.688263,206.066818,298.481995,570.994995,25.266325,0.800000,0.480000,13.052667,...,32.989998,32.559998,73883.296875,102829.898438,247150.000000,214850.000000,85.926274,119.591172,287.435450,249.870550
4,2016-01-01 01:00:00,0.812257,38.688263,193.984100,292.789215,563.804016,25.605207,0.800000,0.480000,10.656666,...,32.719997,32.242664,50396.769531,81683.398438,238550.000000,203300.000000,58.611443,94.997792,277.433650,236.437900


## Reglas de Asociación con el algoritmo Apriori

### COP Carlos

In [4]:
# Dataframe con variables descriptoras
dataCarlos = data[['POTENCIA BOMBA CALOR CARLOS', 'POTENCIA TERMICA BOMBA CALOR CARLOS', 'TEMPERATURA EXTERIOR',
                   'TEMPERATURA SALIDA BOMBA CALOR CARLOS', 'C_O_P BOMBA CALOR CARLOS']]
# Columnas para las etiquetas
dataCarlos["label_potencia"] = ' '
dataCarlos["label_potenciaTermica"] = ' '
dataCarlos["label_temperaturaExterior"] = ' '
dataCarlos["label_temperaturaSalida"] = ' '
dataCarlos["label_COP"] = ' '
dataCarlos.describe()

C:\Users\dag17\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\dag17\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\dag17\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

,POTENCIA BOMBA CALOR CARLOS,POTENCIA TERMICA BOMBA CALOR CARLOS,TEMPERATURA EXTERIOR,TEMPERATURA SALIDA BOMBA CALOR CARLOS,C_O_P BOMBA CALOR CARLOS
count,79285.000000,79285.000000,79285.000000,79285.000000,79285.000000
mean,33.616413,132.154739,15.083257,29.447289,1.603675
std,43.907480,149.917857,11.345877,13.676912,2.059181
min,0.000000,0.000000,-1.400000,0.000000,0.000000
25%,0.480000,9.877769,6.968000,16.940001,0.000000
50%,0.500000,46.868723,13.353139,34.740002,0.000000
75%,67.599998,255.914298,23.717333,42.450001,3.689127
max,164.206665,592.614391,43.799999,47.889999,74.741554


In [5]:
minPotenciaCarlos = 27.5
maxPotenciaCarlos = 67.5
minPotenciaTermicaCarlos = 106
maxPotenciaTermicaCarlos = 255.9
minTempExterior = 12.4
maxTempExterior = 23.7
minTempSalidaCarlos = 28.8
maxTempSalidaCarlos = 42.4

dataCarlosArr = dataCarlos.to_numpy()
size_dataCarlosArr = dataCarlos.shape
filas = size_dataCarlosArr[0]
for i in range(filas):
    if (float(dataCarlosArr[i][0]) > minPotenciaCarlos) and float((dataCarlosArr[i][0]) < maxPotenciaCarlos):
        dataCarlosArr[i][5] = 'potencia buena'
    elif (float(dataCarlosArr[i][0]) <= minPotenciaCarlos):
        dataCarlosArr[i][5] = 'potencia baja'
    elif (float(dataCarlosArr[i][0]) >= maxPotenciaCarlos):
        dataCarlosArr[i][5] = 'potencia alta'

In [6]:
for i in range(filas):
    if (float(dataCarlosArr[i][1]) > minPotenciaTermicaCarlos) and (float(dataCarlosArr[i][1]) < maxPotenciaTermicaCarlos):
        dataCarlosArr[i][6] = 'potencia termica buena'
    elif (float(dataCarlosArr[i][1]) <= minPotenciaTermicaCarlos):
        dataCarlosArr[i][6] = 'potencia termica baja'
    elif (float(dataCarlosArr[i][1]) >= maxPotenciaTermicaCarlos):
        dataCarlosArr[i][6] = 'potencia termica alta'

In [7]:
for i in range(filas):
    if (float(dataCarlosArr[i][2]) > minTempExterior) and (float(dataCarlosArr[i][2]) < maxTempExterior):
        dataCarlosArr[i][7] = 'temp ext buena'
    elif (float(dataCarlosArr[i][2]) <= minTempExterior):
        dataCarlosArr[i][7] = 'temp ext baja'
    elif (float(dataCarlosArr[i][2]) >= maxTempExterior):
        dataCarlosArr[i][7] = 'temp ext alta'

In [8]:
for i in range(filas):
    if (float(dataCarlosArr[i][3]) > minTempExterior) and (float(dataCarlosArr[i][3]) < maxTempExterior):
        dataCarlosArr[i][8] = 'temp sal buena'
    elif (float(dataCarlosArr[i][3]) <= minTempExterior):
        dataCarlosArr[i][8] = 'temp sal baja'
    elif (float(dataCarlosArr[i][3]) >= maxTempExterior):
        dataCarlosArr[i][8] = 'temp sal alta'

In [9]:
for i in range(filas):
    if (float(dataCarlosArr[i][4]) > 3) and (float(dataCarlosArr[i][4]) < 5):
        dataCarlosArr[i][9] = 'cop bueno'
    elif (float(dataCarlosArr[i][4]) <= 3):
        dataCarlosArr[i][9] = 'cop bajo'
    elif (float(dataCarlosArr[i][4]) >= 5):
        dataCarlosArr[i][9] = 'cop alto'

In [10]:
namesCol = dataCarlos.columns.values
dataCarlos_label = pd.DataFrame.from_records(dataCarlosArr, columns = namesCol)
dataCarlosApriori = dataCarlos_label[['label_potencia', 'label_potenciaTermica', 
                                      'label_temperaturaExterior', 'label_temperaturaSalida', 'label_COP']]
dataCarlosApriori

,label_potencia,label_potenciaTermica,label_temperaturaExterior,label_temperaturaSalida,label_COP
0,potencia baja,potencia termica baja,temp ext baja,temp sal alta,cop bajo
1,potencia baja,potencia termica buena,temp ext baja,temp sal alta,cop bajo
2,potencia baja,potencia termica buena,temp ext baja,temp sal alta,cop bajo
3,potencia baja,potencia termica baja,temp ext baja,temp sal alta,cop bajo
4,potencia baja,potencia termica baja,temp ext baja,temp sal alta,cop bajo
5,potencia baja,potencia termica baja,temp ext baja,temp sal alta,cop bajo
6,potencia baja,potencia termica baja,temp ext baja,temp sal alta,cop bajo
7,potencia baja,potencia termica baja,temp ext baja,temp sal alta,cop bajo
8,potencia baja,potencia termica baja,temp ext baja,temp sal alta,cop bajo
9,potencia baja,potencia termica baja,temp ext baja,temp sal alta,cop bajo


In [11]:
def inspect(results):
    rh          = [tuple(result[2][0][0]) for result in results]
    lh          = [tuple(result[2][0][1]) for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(rh, lh, supports, confidences, lifts))

In [12]:
dataCarlosNP = dataCarlosApriori.to_numpy()
transactionsCarlos = list(map(tuple, dataCarlosNP))
rules = apriori(transactionsCarlos, min_support = 0.02, min_confidence = 0.5, min_lift = 3, min_length = 2)
results = list(rules)
# dataframe para ver los datos resultados
resultDataFrame = pd.DataFrame(inspect(results), columns=['rhs','lhs','support','confidence','lift'])
resultDataFrame

,rhs,lhs,support,confidence,lift
0,"(potencia alta,)","(potencia termica alta,)",0.230207,0.918571,3.672847
1,"(potencia buena,)","(potencia termica buena,)",0.124626,0.812248,5.323120
2,"(temp sal baja,)","(temp ext alta,)",0.116605,0.764429,3.050213
3,"(potencia termica alta, cop bajo)","(potencia alta,)",0.028883,0.985794,3.933503
4,"(potencia alta, cop bajo)","(temp ext alta,)",0.038456,0.821169,3.276617
5,"(potencia alta, cop bajo)","(temp sal baja,)",0.039932,0.852680,5.589938
6,"(potencia termica alta, cop bajo)","(temp ext alta,)",0.024065,0.821352,3.277346
7,"(potencia termica alta, cop bajo)","(temp sal baja,)",0.025932,0.885062,5.802230
8,"(potencia alta, cop bueno)","(potencia termica alta,)",0.200996,0.988095,3.950836
9,"(potencia buena, cop bueno)","(potencia termica buena,)",0.106943,0.859939,5.635665


### COP Frio 1

In [13]:
# Dataframe con variables descriptoras
dataFrio1 = data[['POTENCIA GRUPO FRÍO 1', 'POTENCIA TERMICA GRUPO FRIO 1', 'TEMPERATURA EXTERIOR', 'C_O_P MÁQUINA GRUPO FRÍO 1']]
# Columnas para las etiquetas
dataFrio1["label_potencia"] = ' '
dataFrio1["label_potenciaTermica"] = ' '
dataFrio1["label_temperaturaExterior"] = ' '
dataFrio1["label_COP"] = ' '
dataFrio1.describe()

C:\Users\dag17\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\dag17\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\dag17\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

,POTENCIA GRUPO FRÍO 1,POTENCIA TERMICA GRUPO FRIO 1,TEMPERATURA EXTERIOR,C_O_P MÁQUINA GRUPO FRÍO 1
count,79285.000000,79285.000000,79285.000000,79285.000000
mean,20.509645,95.445736,15.083257,3.060061
std,26.984676,117.950837,11.345877,45.129189
min,0.000000,0.000000,-1.400000,0.000000
25%,0.100000,0.000000,6.968000,0.000000
50%,0.100000,22.504057,13.353139,0.000000
75%,38.900002,172.589200,23.717333,4.264040
max,115.366669,2373.624850,43.799999,2521.533447


In [14]:
minPotenciaFrio1 = 11
maxPotenciaFrio1 = 38.9
minPotenciaTermicaFrio1 = 54.6
maxPotenciaTermicaFrio1 = 172.5
minTempExterior = 12.4
maxTempExterior = 23.7

dataFrio1Arr = dataFrio1.to_numpy()
size_dataFrio1Arr = dataFrio1.shape
filas = size_dataFrio1Arr[0]
for i in range(filas):
    if (float(dataFrio1Arr[i][0]) > minPotenciaFrio1) and float((dataFrio1Arr[i][0]) < maxPotenciaFrio1):
        dataFrio1Arr[i][4] = 'potencia buena'
    elif (float(dataFrio1Arr[i][0]) <= minPotenciaFrio1):
        dataFrio1Arr[i][4] = 'potencia baja'
    elif (float(dataFrio1Arr[i][0]) >= maxPotenciaFrio1):
        dataFrio1Arr[i][4] = 'potencia alta'

In [15]:
for i in range(filas):
    if (float(dataFrio1Arr[i][1]) > minPotenciaTermicaFrio1) and float((dataFrio1Arr[i][1]) < maxPotenciaTermicaFrio1):
        dataFrio1Arr[i][5] = 'potencia termica buena'
    elif (float(dataFrio1Arr[i][1]) <= minPotenciaTermicaFrio1):
        dataFrio1Arr[i][5] = 'potencia termica baja'
    elif (float(dataFrio1Arr[i][1]) >= maxPotenciaTermicaFrio1):
        dataFrio1Arr[i][5] = 'potencia termica alta'

In [16]:
for i in range(filas):
    if (float(dataFrio1Arr[i][2]) > minTempExterior) and float((dataFrio1Arr[i][2]) < maxTempExterior):
        dataFrio1Arr[i][6] = 'temp ext buena'
    elif (float(dataFrio1Arr[i][2]) <= minTempExterior):
        dataFrio1Arr[i][6] = 'temp ext baja'
    elif (float(dataFrio1Arr[i][2]) >= maxTempExterior):
        dataFrio1Arr[i][6] = 'temp ext alta'

In [17]:
for i in range(filas):
    if (float(dataFrio1Arr[i][3]) > 3) and float((dataFrio1Arr[i][3]) < 5):
        dataFrio1Arr[i][7] = 'cop bueno'
    elif (float(dataFrio1Arr[i][3]) <= 3):
        dataFrio1Arr[i][7] = 'cop bajo'
    elif (float(dataFrio1Arr[i][3]) >= 5):
        dataFrio1Arr[i][7] = 'cop alto'

In [18]:
namesCol = dataFrio1.columns.values
dataFrio1_label = pd.DataFrame.from_records(dataFrio1Arr, columns = namesCol)
dataFrio1Apriori = dataFrio1_label[['label_potencia', 'label_potenciaTermica', 'label_temperaturaExterior', 'label_COP']]
dataFrio1Apriori

,label_potencia,label_potenciaTermica,label_temperaturaExterior,label_COP
0,potencia baja,potencia termica baja,temp ext baja,cop bajo
1,potencia baja,potencia termica baja,temp ext baja,cop bajo
2,potencia alta,potencia termica alta,temp ext baja,cop bueno
3,potencia alta,potencia termica alta,temp ext baja,cop bueno
4,potencia alta,potencia termica alta,temp ext baja,cop bueno
5,potencia alta,potencia termica alta,temp ext baja,cop bueno
6,potencia alta,potencia termica alta,temp ext baja,cop bueno
7,potencia alta,potencia termica alta,temp ext baja,cop bueno
8,potencia alta,potencia termica alta,temp ext baja,cop bueno
9,potencia buena,potencia termica buena,temp ext baja,cop bueno


In [19]:
dataFrio1NP = dataFrio1Apriori.to_numpy()
transactionsdataFrio1 = list(map(tuple, dataFrio1NP))
rules = apriori(transactionsdataFrio1, min_support = 0.02, min_confidence = 0.5, min_lift = 3, min_length = 2)
results = list(rules)
# dataframe para ver los datos resultados
resultDataFrame = pd.DataFrame(inspect(results), columns=['rhs','lhs','support','confidence','lift'])
resultDataFrame

,rhs,lhs,support,confidence,lift
0,"(potencia alta,)","(potencia termica alta,)",0.197957,0.784162,3.133052
1,"(potencia buena,)","(potencia termica buena,)",0.128196,0.768370,4.118458
2,"(potencia alta, cop bueno)","(potencia termica alta,)",0.172454,0.973029,3.887653
3,"(temp ext alta, cop bueno)","(potencia alta,)",0.074995,0.918018,3.636524
4,"(potencia alta, temp ext baja)","(cop bueno,)",0.067983,0.930433,3.122779
5,"(potencia buena, cop bueno)","(potencia termica buena,)",0.104862,0.873778,4.683444
6,"(temp ext buena, cop bueno)","(potencia buena,)",0.046667,0.574891,3.445740
7,"(temp ext alta, cop bueno)","(potencia termica alta,)",0.077039,0.943029,3.767792
8,"(temp ext alta, potencia termica alta)","(potencia alta,)",0.093347,0.845442,3.349032
9,"(potencia alta, temp ext baja)","(potencia termica alta,)",0.068538,0.938029,3.747813
